# Assembly: load/store operations; arithmetic and bitwise operations; translating assembly code to low-level C code
_COSC 208, Introduction to Computer Systems, 2023-03-22_

## Announcements
* Project 3 due tomorrow @ 11pm
* Office hours tomorrow: 12:30pm-2pm (instead of 9am-10am)

## Outline
* Warm-up
* Mapping assembly code to C code

## Warm-up

_Use the provided ARM64 reference sheet to answer the following questions._
* Q1: _What is the difference between a `neg` instruction and a `mvn` instruction?_

    neg negates a number, whereas mvn performs a bitwise not

<div style="height:4em;"></div>

* Q2: _What is the difference between an `orr` instruction and a `eor` instruction?_

    orr performs a bitwise or, whereas eor performs a bitwise xor

<div style="height:4em;"></div>

* Q3: _What is the difference between a `lsr` instruction and a `ldr` instruction?_

    lsr performs a right shift, whereas ldr copies a value from memory into a register

<div style="height:4em;"></div>

* Q4: _What is the difference between a `add` instruction and a `and` instruction?_

    add performs addition, whereas and performs bitwise and

<div style="height:4em;"></div>

🛑 **STOP here** after completing the above questions; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Mapping assembly code to C code

* Function begin pattern
    * create stack frame by decreasing (`sub`) stack pointer
    * store (`str`) parameters from registers w/x0, w/x1, etc. into the stack in main memory
* Local variable initialization pattern
    * move (mov) constant value into register
    * store (str) register value onto the stack
* Mathematical operation pattern
    * load (`ldr`) first operand from the stack into a register OR move (`mov`) first operand constant into a regster
    * load (`ldr`) second operand from the stack into a register OR move (`mov`) second operand constant into a regster
    * compute
    * store (`str`) resulting value from a register into the stack
* Function return/end pattern
    * load (`ldr`) return value from the stack into the w/x0 register
    * destroy stack frame by increasing (`add`) stack pointer
    * return (`ret`) to caller

* Example

In [ ]:
/* 1*/   #include <stdio.h>
/* 2*/   #include <stdlib.h>
/* 3*/   int dog_years(int birth) {
/* 4*/       int current = 2023;
/* 5*/       int years = current - birth;
/* 6*/       years = years * 7;
/* 7*/       return years;
/* 8*/   }
/* 9*/   int main() {
/*10*/       printf("What year were you born? ");
/*11*/       char year[5];
/*12*/       fgets(year, 5, stdin);
/*13*/       int dog_age = dog_years(atoi(year));
/*14*/       printf("You are %d dog years old\n", dog_age);
/*15*/   }

```
0000000000000914 <dog_years>:
 914:   d10083ff        sub     sp, sp, #0x20  // Line 3: create stack frame
 918:   b9000fe0        str     w0, [sp, #12]  // Line 3: store parameter
 91c:   5280fce0        mov     w0, #0x7e7     // Line 4: assign initial value to local variable
 920:   b9001be0        str     w0, [sp, #24]  //      V
 924:   b9401be1        ldr     w1, [sp, #24]  // Line 5: load first operand 
 928:   b9400fe0        ldr     w0, [sp, #12]  // Line 5: load second operand
 92c:   4b000020        sub     w0, w1, w0     // Line 5: subtract
 930:   b9001fe0        str     w0, [sp, #28]  // Line 5: store result in local variable
 934:   b9401fe1        ldr     w1, [sp, #28]  // Line 6: load first operand
 938:   2a0103e0        mov     w0, w1                 V
 93c:   531d7000        lsl     w0, w0, #3     // Line 6: multiply
 940:   4b010000        sub     w0, w0, w1     //      V
 944:   b9001fe0        str     w0, [sp, #28]  // Line 6: store result in local variable
 948:   b9401fe0        ldr     w0, [sp, #28]  // Line 7: prepare return value
 94c:   910083ff        add     sp, sp, #0x20  // Line 7: destroy stack frame
 950:   d65f03c0        ret                    // Line 7: return to caller
```

```
0000000000000914 <dog_years>:
 914:   d10083ff        sub     sp, sp, #0x20
 918:   b9000fe0        str     w0, [sp, #12]
 91c:   5280fce0        mov     w0, #0x7e7
 920:   b9001be0        str     w0, [sp, #24]
 924:   b9401be1        ldr     w1, [sp, #24]
 928:   b9400fe0        ldr     w0, [sp, #12]
 92c:   4b000020        sub     w0, w1, w0
 930:   b9001fe0        str     w0, [sp, #28]
 934:   b9401fe1        ldr     w1, [sp, #28]
 938:   2a0103e0        mov     w0, w1
 93c:   531d7000        lsl     w0, w0, #3
 940:   4b010000        sub     w0, w0, w1
 944:   b9001fe0        str     w0, [sp, #28]
 948:   b9401fe0        ldr     w0, [sp, #28]
 94c:   910083ff        add     sp, sp, #0x20
 950:   d65f03c0        ret
```

<div style="page-break-after:always;"></div>

* Q5: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

In [2]:
/* 1*/  #include <stdio.h>
/* 2*/  int years_to_double(int rate) {
/* 3*/      int ruleof72 = 72;
/* 4*/      int years = ruleof72 / rate;
/* 5*/      return years;
/* 6*/  }
/* 7*/  int main() {
/* 8*/      int r = 10;
/* 9*/      int y = years_to_double(r);
/*10*/      printf("With an interest rate of %d%% it 
                   will take ~%d years to double your money\n", r, y);
/*11*/  }

With an interest rate of 10% it will take ~7 years to double your money


```
000000000000076c <years_to_double>:
    76c:    d10083ff     sub  sp, sp, #0x20   // 2
    770:    b9000fe0     str  w0, [sp, #12]   // 2
    774:    52800900     mov  w0, #0x48       // 3
    778:    b9001be0     str  w0, [sp, #24]   // 3
    77c:    b9401be1     ldr  w1, [sp, #24]   // 4
    780:    b9400fe0     ldr  w0, [sp, #12]   // 4
    784:    1ac00c20     sdiv w0, w1, w0      // 4
    788:    b9001fe0     str  w0, [sp, #28]   // 4
    78c:    b9401fe0     ldr  w0, [sp, #28]   // 5
    790:    910083ff     add  sp, sp, #0x20   // 5
    794:    d65f03c0     ret                  // 5
```

```
000000000000076c <years_to_double>:
    76c:    d10083ff     sub  sp, sp, #0x20
    770:    b9000fe0     str  w0, [sp, #12]
    774:    52800900     mov  w0, #0x48
    778:    b9001be0     str  w0, [sp, #24]
    77c:    b9401be1     ldr  w1, [sp, #24]
    780:    b9400fe0     ldr  w0, [sp, #12]
    784:    1ac00c20     sdiv w0, w1, w0
    788:    b9001fe0     str  w0, [sp, #28]
    78c:    b9401fe0     ldr  w0, [sp, #28]
    790:    910083ff     add  sp, sp, #0x20
    794:    d65f03c0     ret
```

* Q6: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

In [2]:
/* 1*/  #include <stdio.h>
/* 2*/  long deref(long *p) {
/* 3*/      long v = *p;
/* 4*/      return v;
/* 5*/  }
/* 6*/  int main() {
/* 7*/      long x = 2;
/* 8*/      long y = deref(&x);
/* 9*/      printf("y = %ld\n", y);
/*10*/      return 0;
/*11*/  }

With an interest rate of 10% it will take ~7 years to double your money


```
0000000000000854 <deref>:
 854:   d10083ff    sub sp, sp, #0x20 // 2
 858:   f90007e0    str x0, [sp, #8]  // 2
 85c:   f94007e0    ldr x0, [sp, #8]  // 3
 860:   f9400000    ldr x0, [x0]      // 3
 864:   f9000fe0    str x0, [sp, #24] // 3
 868:   f9400fe0    ldr x0, [sp, #24] // 4
 86c:   910083ff    add sp, sp, #0x20 // 4
 870:   d65f03c0    ret               // 4
```

```
0000000000000854 <deref>:
 854:   d10083ff    sub sp, sp, #0x20
 858:   f90007e0    str x0, [sp, #8]
 85c:   f94007e0    ldr x0, [sp, #8]
 860:   f9400000    ldr x0, [x0]
 864:   f9000fe0    str x0, [sp, #24]
 868:   f9400fe0    ldr x0, [sp, #24]
 86c:   910083ff    add sp, sp, #0x20
 870:   d65f03c0    ret
```

🛑 **STOP here** after completing the above questions; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

The following C program (`operands.c`) has been compiled into assembly:

In [4]:
int operandsA(int a) {
    return a;
}
long operandsB(long b) {
    return b;
}
int operandsC(int *c) {
    return *c;
}
long operandsD(long *d) {
    return *d;
}
int main() {
    operandsA(5);
    operandsB(5);
    int x = 5;
    operandsC(&x);
    long y = 5;
    operandsD(&y);
}

```
00000000000007ec <operandsA>:
    7ec:    d10043ff     sub    sp, sp, #0x10
    7f0:    b9000fe0     str    w0, [sp, #12]
    7f4:    b9400fe0     ldr    w0, [sp, #12]
    7f8:    910043ff     add    sp, sp, #0x10
    7fc:    d65f03c0     ret    
0000000000000800 <operandsB>:
    800:    d10043ff     sub    sp, sp, #0x10
    804:    f90007e0     str    x0, [sp, #8]
    808:    f94007e0     ldr    x0, [sp, #8]
    80c:    910043ff     add    sp, sp, #0x10
    810:    d65f03c0     ret
0000000000000814 <operandsC>:
    814:    d10043ff     sub    sp, sp, #0x10
    818:    f90007e0     str    x0, [sp, #8]
    81c:    f94007e0     ldr    x0, [sp, #8]
    820:    b9400000     ldr    w0, [x0]
    824:    910043ff     add    sp, sp, #0x10
    828:    d65f03c0     ret
000000000000082c <operandsD>:
    82c:    d10043ff     sub    sp, sp, #0x10
    830:    f90007e0     str    x0, [sp, #8]
    834:    f94007e0     ldr    x0, [sp, #8]
    838:    f9400000     ldr    x0, [x0]
    83c:    910043ff     add    sp, sp, #0x10
    840:    d65f03c0     ret
```

<div style="page-break-after:always;"></div>

* Q7: _How does the assembly code for `operandsA` and `operandsB` differ? Why?_

* `operandsA` takes and returns an int, which is 32-bits, whereas `operandsB` takes and returns a long, which is 64-bits, so:
    * `operandsA` uses `w0` while `operandsB` uses `x0`
    * `operandsA` stores the parameter at `sp + 12` while `operandsB` stores the parameter at `sp + 8`

<p style="height:6em;"></p>

* Q8: _How does the assembly code for `operandsB` and `operandsD` differ? Why?_

* `operandsB` takes and returns a long, whereas `operandsD` takes a pointer to a long and returns a long, so:
    * `operandsD` must deference the pointer (`ldr x0, [x0]`) before returning 

<p style="height:6em;"></p>

* Q9: _How does the assembly code for `operandsC` and `operandsD` differ? Why?_

* `operandsC` takes a pointer to an int and returns and int, whereas `operandsD` takes a pointer to a long and returns a long
* both take a memory address (a 64-bit value), which is initially in `x0` and stored at `sp + 8`
* the dereference of the pointer is a 32-bit value in `operandsC` and a 64-bit value in `operandsD`, so the value is loaded into `w0` in `operandsC` and `x0` in `operandsD`

<div style="page-break-after:always;"></div>

## Extra practice

* Q10: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

In [1]:
/* 1*/   #include <stdio.h>
/* 2*/   int seasons_of_love() {
/* 3*/       int dpy = 365;
/* 4*/       int hpd = 24;
/* 5*/       int mph = 60;
/* 6*/       int h = dpy * hpd;
/* 7*/       int m = h * mph;
/* 8*/       return m;
/* 9*/   }
/*10*/  int main() {
/*11*/      int minutes = seasons_of_love();
/*12*/      printf("%d minutes\n", minutes);
/*13*/      printf("%d moments so dear\n", minutes - 600);
/*14*/      printf("%d minutes\n", minutes);
/*15*/      printf("How do you measure? Measure a year?\n");
/*16*/  }

525600 minutes
525000 moments so dear
525600 minutes
How do you measure? Measure a year?


```
00000000000007ac <seasons_of_love>:
    7ac:   d10083ff        sub     sp, sp, #0x20    // Line 2   Create stack frame
    7b0:   52802da0        mov     w0, #0x16d       // Line 3   Local variable (dpy) initialization 
    7b4:   b9000fe0        str     w0, [sp, #12]    //   V
    7b8:   52800300        mov     w0, #0x18        // Line 4   Local variable (hpd) initialization
    7bc:   b90013e0        str     w0, [sp, #16]    //   V
    7c0:   52800780        mov     w0, #0x3c        // Line 5   Local variable (mph) initialization
    7c4:   b90017e0        str     w0, [sp, #20]    //   V
    7c8:   b9400fe1        ldr     w1, [sp, #12]    // Line 6   Mathematical operation
    7cc:   b94013e0        ldr     w0, [sp, #16]    //   |
    7d0:   1b007c20        mul     w0, w1, w0       //   |
    7d4:   b9001be0        str     w0, [sp, #24]    //   V
    7d8:   b9401be1        ldr     w1, [sp, #24]    // Line 7   Mathematical operation
    7dc:   b94017e0        ldr     w0, [sp, #20]    //   |
    7e0:   1b007c20        mul     w0, w1, w0       //   |
    7e4:   b9001fe0        str     w0, [sp, #28]    //   V
    7e8:   b9401fe0        ldr     w0, [sp, #28]    // Line 8   Function return/end
    7ec:   910083ff        add     sp, sp, #0x20    //   |
    7f0:   d65f03c0        ret                      //   V
```

```
00000000000007ac <seasons_of_love>:
    7ac:   d10083ff        sub     sp, sp, #0x20
    7b0:   52802da0        mov     w0, #0x16d  
    7b4:   b9000fe0        str     w0, [sp, #12]
    7b8:   52800300        mov     w0, #0x18   
    7bc:   b90013e0        str     w0, [sp, #16]
    7c0:   52800780        mov     w0, #0x3c   
    7c4:   b90017e0        str     w0, [sp, #20]
    7c8:   b9400fe1        ldr     w1, [sp, #12]
    7cc:   b94013e0        ldr     w0, [sp, #16]
    7d0:   1b007c20        mul     w0, w1, w0
    7d4:   b9001be0        str     w0, [sp, #24]
    7d8:   b9401be1        ldr     w1, [sp, #24]
    7dc:   b94017e0        ldr     w0, [sp, #20]
    7e0:   1b007c20        mul     w0, w1, w0
    7e4:   b9001fe0        str     w0, [sp, #28]
    7e8:   b9401fe0        ldr     w0, [sp, #28]
    7ec:   910083ff        add     sp, sp, #0x20
    7f0:   d65f03c0        ret
```